## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
#讀取資料集
from sklearn import datasets, metrics
digits = datasets.load_digits()

In [2]:
#切分訓練集/測試集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=4)

In [3]:
#建模-梯度提升機(分類)
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=7)

In [4]:
#訓練模型
gbc.fit(x_train, y_train)

GradientBoostingClassifier(random_state=7)

In [5]:
#預測測試集
y_pred = gbc.predict(x_test)

In [6]:
#準確度計算
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
##Accuracy:  0.9644444444444444

Accuracy:  0.9644444444444444


In [7]:
#???有用到KFold
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

# 設定要訓練的超參數組合
#???參數組合如何設置
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
#因為是分類故將scoring="neg_mean_squared_error"改成"accuracy"
grid_search = GridSearchCV(gbc, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.8min finished


In [8]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.968812 using {'max_depth': 3, 'n_estimators': 300}


In [9]:
# 使用最佳參數重新建立模型
gbc_bestparam = GradientBoostingClassifier(max_depth = grid_result.best_params_['max_depth'],
                                           n_estimators = grid_result.best_params_['n_estimators'])

# 訓練模型
gbc_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = gbc_bestparam.predict(x_test)

In [10]:
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
##Accuracy:  0.9711111111111111

Accuracy:  0.9711111111111111
